<a href="https://colab.research.google.com/github/rg-io/HMM/blob/master/HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
''' main cell '''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 21 09:02:49 2020

@author: ataraxis
"""

from scipy.io import loadmat as load
import numpy as np
from flat_start import flat_start
from phn_o_const import phn_o_const
from train_model import train_model
c0 = load('c0_coded_sep_lrg_nrm.mat')
c0_obs = c0['c0_new'] #c0_obs is a 1 x 10 matrix

q = 4 #number of states
utter = 0

maxiter = 15
N = 4
M = 32
	
A = np.zeros((N,N), dtype = np.double)
B = np.zeros((N,M), dtype = np.double)
P = np.ones((N,1), dtype = np.double)	
	
P = P/sum(P)
log_lh_arr = []
A_big = {}
B_big = {}

for i in range(0, N-1):
	A[i,i] = 0.5
	A[i,i+1] =0.5
	
for i in range(0, q):
    A_big[i] = A
	
for i in range(0, N):
    for j in range(0, M):
        B[i,j] = 1/M

for i in range(0, M):
    B[N-1, i] = 0

for i in range(0, q):
        B_big[i] = B
    

phn_sgmt = flat_start(c0_obs, q) #function call


phn_sgmt_mat = np.zeros((10,q), dtype = object)
for i in range(0, 10):
     for j in range(0, q):
         phn_sgmt_mat[i,j] = phn_sgmt[i,j]
# Had to make phn_sgmt_mat to mimic the cells in matlab

    
#train

lh_arr = np.zeros((q, 1)) #4 X 1 matrix
lh_mat = np.zeros((maxiter, q)) #15 X 4 matrix
phn_O = np.zeros((1), dtype = object)
A_new = np.zeros((N,N), dtype = np.double)
B_new = np.zeros((N,M), dtype = np.double)
P_new = np.ones((N,1), dtype = np.double)
log_lh = 0

for i in range(0, maxiter):
    for j in range(0, q):
        phn_O = phn_o_const(phn_sgmt_mat, j)
        #print(30081995)
        A_new, B_new, P_new, log_lh = train_model(phn_O,A_big[j],B_big[j], P)
        A_new


In [ ]:
def backward(obs,A,B,P,scale):
    
    # obs : T*1 --> obs vector
    # A = N*N np --> transition matrix
    # B = N * M --> emission matrix ( M = 32)
    # P = N*1 --> prior probabiities
    # scale = T*1 --> to prevent underflow of beta 
    
    N = A.shape[0]
    T = obs.shape[0]
    beta = np.zeros((N,T),np.float64)
    beta_hat = np.zeros(beta.shape,np.float64)
    
    beta[N-1,T-1] = 1;
    
    for i in range(N-1):
        beta[i,T-1] = A[i,N-1] # check
    
    beta_hat[:,T-1] = beta[:,T-1]
    
    for t in range(T-2,-1,-1):
        for j in range(N-2,-1,-1):
            
            temp = 0
            for i in range(N):
                if i % 4 == 0 and i != 0:
                    temp += A[j,i]*beta_hat[i,t]
                else:
                    temp += A[j,i] * beta_hat[i,t+1]*B[i,obs[t+1,0]]
            
            beta[j,t] = temp
            beta_hat[j,t] = temp;
        
        beta_hat[:,t] = beta[:,t]/scale[t+1,0]
        
    return beta_hat,beta
     
    


In [8]:
'''forward cell'''
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Aug  2 19:01:52 2020

@author: ataraxis
"""

import numpy as np


def fwdfun(obs, A, B, Pt):
    N = len(A) #here A will be a dictionary with transition probabilities
    T = len(obs) #here T is the time #E 
    alpha = np.zeros((N,T), dtype = np.double)
    alpha_hat = alpha
    scale = np.zeros((T,1), dtype = np.double)
    
    for i in range(0, N):
        alpha[i, 0] = Pt[i] * B[i, obs[0,0]]
        
    val = 0
        
    for i in range(0, N):
        val += alpha[i,0]
    scale[0,0] = val
    
    if (scale[0,0] == 0):
        scale[0,0]  = 1

    for i in range(0, alpha.shape(0)):
        alpha_hat[i,0] =  alpha[i,0]/scale[0,0]
        
    for t in range(1, T):
        for j in range(0, N):
            temp = 0
            for i in range(0, N):
                if(j%4 ==0):
                    temp = temp + A[i,j]*alpha_hat[i,t]
                else:
                    temp = temp + A[i,j]*alpha_hat[i, t-i]*B[j, obs[t,0]]
            alpha[j,t] = temp
            alpha_hat[j,t] = temp
     
        scale[t, 0] = np.sum(alpha,0)[t]
        if (scale[t,0] == 0):
            scale[t,0] = 1
        for i in range(0, alpha.shape(0)):
            alpha_hat[i,t] =  alpha[i,t]/scale[t,0]
    lhood = np.sum(alpha, 0)[T-1]
    
    return (alpha_hat, scale, lhood)
        
        
    
    
    